In [6]:
%matplotlib inline

In [7]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import os.path
import numpy as np

# This is empirically verified, DO NOT MODIFY
PEAK_REL_AMP = 0.563
PEAK_REL_AMP_DB = 20. * np.log10(PEAK_REL_AMP)

# Verify WAV stimuli for MEG experiment

WAV-files used in the MEG should

* be sampled at 44.1 kHz
* be saved as 16-bit integer precision

If this notebook runs without error, that is the case. In addition, the final plot can be inspected to see the largest absolute amplitude of the stimulus set used. Due to mismatches between the input- and output-levels of the various components in the audio-stream:

 > __the peak amplitude should never exceed 0.563 (-4.99 dB), _i.e._, 56.3% of maximum volume output__
 
 ## File layout
 
Place all the files in a single folder, and set the variable `wavdir` (below) to point to it.

In [8]:
proj_name = 'MEG_EEG-Training'
wavdir = os.path.join('/aux', proj_name, 'wav_stimuli')
wavdir = '/Users/cjb/tmp/rasha'

In [9]:
if not os.path.exists(os.path.expanduser(wavdir)):
    raise IOError('Invalid wav-directory: {:s}'.format(wavdir))

In [10]:
import sys
sys.path.insert(0, '/Users/cjb/src/git/meeg-cfin/stimulation/utilities')
from wavhelpers import (list_wavs_in_dir, get_wav, wavlist_to_wavarr)
Fs = 44100.  # get_wav hard-coded to raise error if not 44.1 kHz

### Get a list of pathnames to the WAV-files

In [11]:
wavnames = list_wavs_in_dir(wavdir)

### Read each WAV into memory

Each file is read into a list of 2D-arrays, after which we know the duration of the longest stimulus. The function `wavlist_to_wavarr` then zero-pads each array to have equal duration, and returns a 3D array with dimensions `n_files x n_channels x n_timepoints`, where `n_channels` is either 1 (mono) or 2 (stereo).

In [ ]:
wavlist = []
for fname in wavnames:
    wavlist += [get_wav(fname)]
wavarr = wavlist_to_wavarr(wavlist)
del wavlist

# now data is guaranteed to be 16 bit short
maxVal = 2**15 - 1

### Plot every N:th WAV-file

Currently plots only left channel.

In [ ]:
plot_every_nth = 2
timepoints = np.arange(0, wavarr.shape[2]/Fs, 1./Fs)
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
for nwav in range(0, wavarr.shape[0], plot_every_nth):
    ax.plot(timepoints, wavarr[nwav,0,:].astype(float) / np.float(maxVal),
            label=os.path.basename(wavnames[nwav]))
ax.legend()

## The stimulus envelope (peak value)

In [ ]:
# max_env = np.abs(wavarr).max(axis=0).astype(float) / np.float(maxVal)
max_env = 20. * np.log10(np.abs(wavarr).max(axis=0).astype(float) /
                         np.float(maxVal))
max_stim_env = max_env.max()
print('The maximum stimulus envelope is {:.3f} '
      '({:.3f} dB)'.format(10**(max_stim_env / 20.), max_stim_env))

## A plot of the peak range for all stimuli

This plots the peak absolute values of the stimuli, in dB relative to the maximum possible in the 16-bit wav file.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
for ww in range(wavarr.shape[0]):
    ax.plot(timepoints, 
            20. * np.log10(np.abs(wavarr[ww, 0, :]).astype(float) /
                           np.float(maxVal)),
            label=os.path.basename(wavnames[ww]))
ax.set_ylim(-9., 0.)
ax.legend()
_ = ax.set_yticks([0., max_stim_env, -3., -6., -9.])

## What should I do with my stimuli?

Now that you know the peak envelope of your stimulus set, we can calculate whether and how much additional dampening of the stimuli is required for no clipping to occur.

In [ ]:
att_factor = PEAK_REL_AMP / 10**(max_stim_env / 20.)
if max_stim_env > PEAK_REL_AMP_DB:
    print('You should attenuate your stimuli by '
          'a factor of {:.3f} ({:.3f} dB)'.format(att_factor,
                                                  20 * np.log10(att_factor)))
else:
    print('Your stimuli are sufficiently damped, '
          'you are using {:.1f}% of the range.'.format(100./att_factor))